In [1]:
import csv
import gzip
import os
import scipy.io


In [2]:
pwd

'/Users/lidiayung/github/notebooks/GBM'

In [3]:

# define MEX directory
matrix_dir = "/Users/lidiayung/project/resource/GSM5319572_SF4324/"
# read in MEX format matrix as table
mat = scipy.io.mmread(os.path.join(matrix_dir, "matrix.mtx.gz"))
 
# list of transcript ids, e.g. 'ENSG00000243485'
features_path = os.path.join(matrix_dir, "features.tsv.gz")
#feature_ids = [row[0] for row in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]
 
# list of gene names, e.g. 'MIR1302-2HG'
gene_names = [row[0] for row in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]
 
# list of feature_types, e.g. 'Gene Expression'
feature_types = [row[0] for row in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]
barcodes_path = os.path.join(matrix_dir, "barcodes.tsv.gz")
barcodes = [row[0] for row in csv.reader(gzip.open(barcodes_path, mode="rt"), delimiter="\t")]

In [4]:
import pandas as pd
 
# transform table to pandas dataframe and label rows and columns
matrix = pd.DataFrame.sparse.from_spmatrix(mat)
matrix.columns = barcodes
#matrix.insert(loc=0, column="feature_id", value=feature_ids)
#matrix.insert(loc=0, column="feature_type", value=feature_types)
matrix=matrix.transpose() 
# display matrix
matrix.columns = gene_names
print(matrix.head())
# save the table as a CSV (note the CSV will be a very large file)
#matrix.to_csv("mex_matrix.csv", index=False)

                    RP11-34P13.3  FAM138A  OR4F5  RP11-34P13.7  RP11-34P13.8  \
AAACCCAAGAAGGTAG-1             0        0      0             0             0   
AAACCCACAGAGTGAC-1             0        0      0             0             0   
AAACCCAGTCATGGCC-1             0        0      0             0             0   
AAACCCATCGTTACCC-1             0        0      0             0             0   
AAACCCATCTCCACTG-1             0        0      0             0             0   

                    RP11-34P13.14  RP11-34P13.9  FO538757.3  FO538757.2  \
AAACCCAAGAAGGTAG-1              0             0           0           1   
AAACCCACAGAGTGAC-1              0             0           0           1   
AAACCCAGTCATGGCC-1              0             0           0           1   
AAACCCATCGTTACCC-1              0             0           0           0   
AAACCCATCTCCACTG-1              0             0           0           0   

                    AP006222.2  ...  AC007325.2  BX072566.1  AL35482

In [5]:
matrix.shape

(3688, 33694)

In [6]:
newlist =[]
for x in barcodes:
    newlist.append(x.replace('-1', ''))


In [7]:
matrix["Barcode"]=newlist

In [8]:
file = "/Users/lidiayung/project/resource/GSM5319572_SF4324/GSE174554_Tumor_normal_metadata.txt"

In [9]:
metadata= pd.read_csv(file,sep=' ')
metadata.head()

,Sample#,Barcode,Tumor_Normal_annotation
0,SF10022,CTATCTAAGCAAGCCA,Tumor
1,SF10022,AAACCCAGTCTACGAT,Normal
2,SF10022,AAAGGGCTCACCCTGT,Normal
3,SF10022,AACAACCAGACCCGCT,Normal
4,SF10022,AACAAGAGTGTAAACA,Normal


In [10]:
metadata.groupby("Sample#")["Tumor_Normal_annotation"].count()

Sample#
SF10022      3082
SF10099       568
SF10099v2     234
SF10108      4446
SF10127      1052
             ... 
SF9715v2     3731
SF9791v2     1608
SF9798       4051
SF9871       3602
SF9962       3468
Name: Tumor_Normal_annotation, Length: 100, dtype: int64

In [54]:
sum = metadata.groupby("Sample#")["Tumor_Normal_annotation"].count()

In [58]:
pd.DataFrame(sum)

,Tumor_Normal_annotation
Sample#,
SF10022,3082
SF10099,568
SF10099v2,234
SF10108,4446
SF10127,1052
...,...
SF9715v2,3731
SF9791v2,1608
SF9798,4051


In [60]:
tumor_count=[]
for i in metadata["Sample#"]:
    tumor_count = metadata[[[metadata["Sample#"]== i]|metadata["Tumor_Normal_annotation"]=="Tumor"]]
    tumor_count.append(tumor_count)

ValueError: Arrays were different lengths: 254288 vs 1

In [49]:
table= pd.pivot_table(metadata,index=['Sample#','Tumor_Normal_annotation'],aggfunc='count')

In [30]:
tumor_total = metadata[metadata['Tumor_Normal_annotation'] == 'Tumor']

In [31]:
tumor_total

,Sample#,Barcode,Tumor_Normal_annotation
0,SF10022,CTATCTAAGCAAGCCA,Tumor
172,SF10022,TCACTATAGATGCGAC,Tumor
185,SF10022,AGGACGAGTGTGCTTA,Tumor
186,SF10022,ATAGAGATCCCTGTTG,Tumor
187,SF10022,CCAATTTAGGCTTTCA,Tumor
...,...,...,...
254282,SF9791v2,TTTATGCTCTCCTGAC,Tumor
254283,SF9791v2,TTTCAGTCATCTTCGC,Tumor
254284,SF9791v2,TTTCGATAGTCATAGA,Tumor
254285,SF9791v2,TTTGGAGCACTGTCGG,Tumor


In [47]:
tumor_percentage=[]
for i in metadata['Sample#']:
    tumor_total = metadata[metadata['Tumor_Normal_annotation'] == 'Tumor']
    tumor_count = tumor_total[tumor_total["Sample#"]== i].count()
    total=metadata[metadata["Sample#"]== i].count()
    percentage = tumor_count/total*100
    tumor_percentage.append(percentage)

KeyboardInterrupt: 

In [38]:
tumor_count = tumor_total[tumor_total["Sample#"]== "SF9791v2"].count()
print(tumor_count)

Sample#                    1339
Barcode                    1339
Tumor_Normal_annotation    1339
dtype: int64


In [27]:

    total=metadata[i].count()
    percentage = tumor_count/total*100
    tumor_percentage.append(i,percentage)
for 
tumor_count=metadata[metadata['Tumor_Normal_annotation']=='Tumor']['Sample#'].count()

In [53]:
table['count']

KeyError: 'count'

In [52]:
table.describe

<bound method NDFrame.describe of                                    Barcode
Sample#   Tumor_Normal_annotation         
SF10022   Normal                      2490
          Tumor                        592
SF10099   Normal                       119
          Tumor                        449
SF10099v2 Normal                       232
...                                    ...
SF9798    Tumor                       3850
SF9871    Normal                      3596
          Tumor                          6
SF9962    Normal                       808
          Tumor                       2660

[200 rows x 1 columns]>

In [46]:
percentage=[]
for i,h in table['count']:
    percentage = h/(i+h)*100
    percentage.append(percentage)
print(percentage)

ValueError: too many values to unpack (expected 2)

In [13]:
table.to_csv("tumor_percentage_table.csv",sep=',',index=True)

In [14]:
tumor_percentage=[]
for i in metadata['Sample#']:
    tumor_total = meta[metadata['Tumor_Normal_annotation'] == 'Tumor']
    
    [i].sum()
    total=metadata[i].sum()
    percentage = tumor_total/total*100
    tumor_percentage.append(i,percentage)

SyntaxError: unmatched ']' (3702138833.py, line 5)

In [ ]:
table['% Tumor'] = (table['Tumor_Normal_annotation']/table['Barcode'].sum())*100

In [ ]:
metadata.shape

In [ ]:
SF4324 = metadata[metadata['Sample#']=='SF4324v2']
SF4324= SF4324[SF4324['Tumor_Normal_annotation']=="Tumor"]

In [ ]:
SF4324.shape

In [ ]:
merge = matrix.merge(SF4324, on="Barcode")
data = matrix[matrix['Barcode'].isin(merge["Barcode"])]
data.shape

In [ ]:
data.head()

In [ ]:
#data.to_csv("tumor_matrix.csv", index=True)